In [ ]:
import tensorflow as tf
from tensorflow import keras
import pickle

title_model = keras.models.load_model('/content/drive/MyDrive/title_classifier/content/title_classifier')
stats_clf = pickle.load(open('/content/drive/MyDrive/Stats_Estimator_Ensemble.sav', 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking c

In [ ]:
import pandas as pd

clickBait = pd.read_csv('/content/drive/MyDrive/DataCollection/clickbait.csv', error_bad_lines=False)
notClickBait = pd.read_csv('/content/drive/MyDrive/DataCollection/notClickbait.csv', error_bad_lines=False)

for like in clickBait["Likes"]:
  clickBait["Dislike-to-like-Ratio"] = clickBait["Dislikes"] / like

for like in notClickBait["Likes"]:
  notClickBait["Dislike-to-like-Ratio"] = notClickBait["Dislikes"] / like

del notClickBait["Favorites"]
del notClickBait["ID"]
del clickBait["Favorites"]
del clickBait["ID"]

clickBait["isClickBait"] = 1
notClickBait["isClickBait"] = 0

data = pd.concat([clickBait, notClickBait])
data.sample(frac=1, random_state=42)

,Video Title,Views,Likes,Dislikes,Dislike-to-like-Ratio,isClickBait
95,SHE IS PREGNANT!!,6021486,217166,4904,0.002575,1
15,BRENT AND I PRANK OUR MOM!!,15114182,375282,7265,0.003814,1
30,kid shoots dad after he takes PS5..,29079,2177,368,0.000193,1
57,DaBaby - Wockesha (Freestyle) [Official Video],1823136,137724,3211,0.233970,0
27,Qualifying Highlights 2021 Dutch Grand Prix,3304392,81058,930,0.067765,0
...,...,...,...,...,...,...
5,Turning cotton balls into cotton candy,1609573,131024,704,0.051297,0
14,"Cutting Peoples Earphones, Then Giving Them Ai...",43680379,1297208,40894,0.021471,1
92,DONT PLAY GTA 5 AT 3:00 AM.. (scary),2974379,78932,10362,0.005440,1
78,OFFICIAL SCIENTIST Mod in Among Us,3607286,238848,3356,0.244535,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, data["isClickBait"], test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler

stats = X_train[["Views", "Likes", "Dislikes"]]
scaler = StandardScaler()
scaler.fit(stats)
stats_scaled = pd.DataFrame(scaler.transform(stats))
X_train[["Views", "Likes", "Dislikes"]] = stats_scaled

stats2 = X_test[["Views", "Likes", "Dislikes"]]
scaler2 = StandardScaler()
scaler2.fit(stats2)
stats_scaled = pd.DataFrame(scaler2.transform(stats2))
X_test[["Views", "Likes", "Dislikes"]] = stats_scaled
X_test.fillna(0,inplace=True)
X_test.head(10)

,Video Title,Views,Likes,Dislikes,Dislike-to-like-Ratio,isClickBait
37,Lil Duval - Sexy (Official Video) ft. Boosie B...,0.101539,-0.318039,-0.180197,0.036505,0
13,30 BEST PRANKS AND FUNNY TRICKS FOR YOUR FRIENDS,-0.500315,-0.433757,-0.298383,0.002449,1
9,"Food Theory M&Ms Melt In Your Hand, LIE To You...",-0.453634,-0.334265,-0.261344,0.124162,0
6,10 Real Life GIANTS,-0.498131,-0.432298,-0.298654,0.003417,1
14,"Cutting Peoples Earphones, Then Giving Them Ai...",0.074328,-0.362898,-0.110387,0.021471,1
4,10 Mythical CREATURES That Actually Existed,2.638681,2.715276,0.623711,0.001026,1
60,PREGNANT SEX PRANK!,0.000000,0.000000,0.000000,0.000028,1
20,DMing 100 YouTubers Asking For A Dare! (i regr...,-0.436584,-0.116376,-0.289169,0.003012,1
84,How To Win 1v2's,0.000000,0.000000,0.000000,0.042480,0
91,Official Moving Day!,0.000000,0.000000,0.000000,0.124308,0


In [ ]:
import numpy as np

def logits_to_proba(values):
  """Converts an array of logit values into an array of probabilities using the equation found here:
  https://www.montana.edu/rotella/documents/502/Prob_odds_log-odds.pdf"""
  probs = []
  for v in values:
    probs.append((np.exp(v))/(1 + np.exp(v)))
  
  return probs

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_title = title_model.predict(X_test["Video Title"])
y_pred_stats = stats_clf.predict(X_test[["Views", "Likes","Dislikes","Dislike-to-like-Ratio"]])


title_predictions = []

for p in logits_to_proba(y_pred_title):
  if( p > 0.5):
    title_predictions.append(1)
  else :
    title_predictions.append(0)

print("Title: " + str(accuracy_score(y_test, title_predictions)))
print("Stats: " + str(accuracy_score(y_test, y_pred_stats)))

Title: 0.975609756097561
Stats: 0.8536585365853658


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


Paste this in the backend code:

In [ ]:
import pickle

class YouTube_Classifier_Ensemble():
     """A custom Scikit-Learn based Ensemble classifier using a classifier for video titles and another for video statistics for the goal of YouTube Video 
     Clickbait Classification"""

     def __init__(self, title_filepath, stats_filepath):
       self.title = keras.models.load_model(title_filepath)
       self.stats = pickle.load(open(stats_filepath, 'rb'))

     def predict_probabilities(self, title_data, stats_data):
      """Returns an array of probabilities of whether the title and stats of a YouTube video are clickbait"""
      y_pred_title = self.title.predict(title_data)
      
      y_pred_stats = self.stats.predict_proba(stats_data)

      title_predictions = []

      for p in logits_to_proba(y_pred_title):
        if( p > 0.5):
          title_predictions.append(1)
        else :
          title_predictions.append(0)


      final_predictions = []


      return final_predictions

In [ ]:
clf = YouTube_Classifier_Ensemble('/content/drive/MyDrive/title_classifier/content/title_classifier', '/content/drive/MyDrive/Stats_Estimator_Ensemble.sav')

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.22.2.post1 when using version 1.0.1. This might lead to breaking c

In [ ]:
print(clf.predict_probabilities(X_test["Video Title"], X_test[["Views", "Likes","Dislikes","Dislike-to-like-Ratio"]]))

[array([-3.42615245, -3.46247842]), array([3.08791648, 3.33372201]), array([-2.95893085, -3.13373387]), array([3.09698986, 3.34272743]), array([2.90104381, 3.05858621]), array([3.10549772, 3.27877486]), array([3.04798977, 3.21694572]), array([2.81976409, 2.95982461]), array([-3.12163434, -3.28037563]), array([-2.53627417, -2.71440723]), array([2.89722531, 3.06618126]), array([-3.2244539 , -3.40258696]), array([-3.51460714, -3.35565119]), array([3.15321451, 3.39655203]), array([0.07046855, 0.2244245 ]), array([0.3594068 , 0.18899885]), array([3.1381887 , 3.30881132]), array([-3.42923772, -3.55647147]), array([-3.20154997, -3.39185098]), array([3.1964328 , 3.02227619]), array([-2.94840729, -3.05840289]), array([-3.46562897, -3.64376129]), array([3.13703541, 3.30307528]), array([3.15060216, 3.29622478]), array([-3.27017803, -3.11234741]), array([3.06338928, 3.08928063]), array([3.12244446, 3.36690013]), array([-2.71637104, -2.89450409]), array([-3.28626801, -3.41809343]), array([-3.229781

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


def logits_to_proba(values):
  """Converts an array of logit values into an array of probabilities using the equation found here:
  https://www.montana.edu/rotella/documents/502/Prob_odds_log-odds.pdf"""
  probs = []
  for v in values:
    probs.append((np.exp(v))/(1 + np.exp(v)))
  
  return probs


def preprocess_data(clickBait, notClickBait):
  """Preprocesses the data from the clickBait and notClickBait csv's, and returns an array with the ready X_train, y_train, X_test, and y_test to be used"""

  for like in clickBait["Likes"]:
    clickBait["Dislike-to-like-Ratio"] = clickBait["Dislikes"] / like

  for like in notClickBait["Likes"]:
    notClickBait["Dislike-to-like-Ratio"] = notClickBait["Dislikes"] / like

  del notClickBait["Favorites"]
  del notClickBait["ID"]
  del clickBait["Favorites"]
  del clickBait["ID"]

  clickBait["isClickBait"] = 1
  notClickBait["isClickBait"] = 0

  data = pd.concat([clickBait, notClickBait])
  data.sample(frac=1, random_state=42)


  X_train, X_test, y_train, y_test = train_test_split(data, data["isClickBait"], test_size=0.2)


  stats = X_train[["Views", "Likes", "Dislikes"]]
  scaler = StandardScaler()
  scaler.fit(stats)
  stats_scaled = pd.DataFrame(scaler.transform(stats))
  X_train[["Views", "Likes", "Dislikes"]] = stats_scaled

  stats2 = X_test[["Views", "Likes", "Dislikes"]]
  scaler2 = StandardScaler()
  scaler2.fit(stats2)
  stats_scaled = pd.DataFrame(scaler2.transform(stats2))
  X_test[["Views", "Likes", "Dislikes"]] = stats_scaled
  X_test.fillna(0,inplace=True)
  X_test.head(10)

  return [X_train, X_test, y_train, y_test]